# Spark MLlib
## Lab assignment: Example with the MLlib

The aim of this notebook is to play with the MLlib of Apache Spark to create a Machine Learning pipeline that preprocess a dataset, train a model and make predictions. In particular, we are going to deal with a regression problem. The content of this lab has been inspired by the sample provided in the [DataBricks documentation](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/2854662143668609/2084788691983918/6837869239396014/latest.html).


## Appliances Energy Prediction

**Data**: This dataset contains Energy consumption from appliances at 10 min resolution for about 4.5 months. The house temperature and humidity conditions were monitored with a wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes. Weather from the nearest airport weather station (Chievres Airport, Belgium) was also logged. This dataset is from [Candanedo et al](http://dx.doi.org/10.1016/j.enbuild.2017.01.083) and is hosted by the UCI Machine Learning Repository. [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction).


**Goal**: We want to learn to predict appliances' energy consumption based on weather information. It would also be nice to know which input features are the most relevant to make predictions.

**Approach**: We will use Spark ML Pipelines, which help users piece together parts of a workflow such as feature processing and model training. We will also demonstrate model selection (a.k.a. hyperparameter tuning) using Cross Validation in order to fine-tune and improve our ML model.

## Submission and marking criteria

You should complete this notebook and add your solutions to it. When you are done, rename your completed notebook as `ex04.ipynb`. 

Important notes:
- The **group leader** must submit the `ex04.ipynb` file on Moodle.
- **Each member of the group** must complete the peer review survey and their contribution statement using this [link](https://forms.office.com/Pages/ResponsePage.aspx?id=7qe9Z4D970GskTWEGCkKHjZupmfSK6JKqlvGZrucaoBURFJJWllYWVhQS09PMFNBVzlCT05JUjM4VCQlQCN0PWcu). **You can only submit this survey ONCE**.
- This lab is marked out of 100 marks, and each section is allocated a number of marks that are indicated below.
- The marking will be focused on the efficiency of the solution and its functionality. Minor mistakes will deduct marks from each exercise.

- **Submission deadline: 8th April 2024 at 3pm**

The first thing we need to do to start working with Spark is to initialize the `SparkSession`. We will also import a few libraries we will use. *Remember if you are using Databricks that `spark` and `sc` are already available to you and don't need initializing.* 

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("MLlib lab") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import Row

# Helper function to test the correctness of the solutions
def test(var, val, msg=""):
    print("1 test passed.") if var == val else print("1 test failed. " + msg)

## Load and understand the data  [10 marks]

We begin by loading the data, which is in Comma-Separated Value (CSV) format. For that, you should use `spark.read` to read the file. Then, you should also cache the data so that we only read it from disk once.

In [6]:
# <FILL-IN WITH YOUR CODE>
df = spark.read.format("csv").option("header", "true").load("data/energydata_complete.csv")
# Cache your DataFrame
# <FILL-IN WITH YOUR CODE>
df.cache()

DataFrame[date: string, Appliances: string, lights: string, T1: string, RH_1: string, T2: string, RH_2: string, T3: string, RH_3: string, T4: string, RH_4: string, T5: string, RH_5: string, T6: string, RH_6: string, T7: string, RH_7: string, T8: string, RH_8: string, T9: string, RH_9: string, T_out: string, Press_mm_hg: string, RH_out: string, Windspeed: string, Visibility: string, Tdewpoint: string, rv1: string, rv2: string]

In [7]:
test(df.count(), 19735, 'Incorrect number of rows')
test(df.is_cached, True, 'df not cached')

1 test passed.
1 test passed.


#### Data description

From the UCI ML Repository description, we know that the columns have the following meanings.

**Attribute information**:
```
date time year-month-day hour:minute:second
Appliances, energy use in Wh
lights, energy use of light fixtures in the house in Wh
T1, Temperature in kitchen area, in Celsius
RH_1, Humidity in kitchen area, in %
T2, Temperature in living room area, in Celsius
RH_2, Humidity in living room area, in %
T3, Temperature in laundry room area
RH_3, Humidity in laundry room area, in %
T4, Temperature in office room, in Celsius
RH_4, Humidity in office room, in %
T5, Temperature in bathroom, in Celsius
RH_5, Humidity in bathroom, in %
T6, Temperature outside the building (north side), in Celsius
RH_6, Humidity outside the building (north side), in %
T7, Temperature in ironing room , in Celsius
RH_7, Humidity in ironing room, in %
T8, Temperature in teenager room 2, in Celsius
RH_8, Humidity in teenager room 2, in %
T9, Temperature in parents room, in Celsius
RH_9, Humidity in parents room, in %
To, Temperature outside (from Chievres weather station), in Celsius
Pressure (from Chievres weather station), in mm Hg
RH_out, Humidity outside (from Chievres weather station), in %
Wind speed (from Chievres weather station), in m/s
Visibility (from Chievres weather station), in km
Tdewpoint (from Chievres weather station), Â°C
rv1, Random variable 1, nondimensional
rv2, Random variable 2, nondimensional
```

**The target variable is the energy use of the Appliances.**

For now, we will leave the two variables `rv1` and `rv2` in our dataset, to see if they are affecting much our methods, then we can try to remove them and see if we improve the results.

Use `show` to visualise the data. Be careful not to show the entire data frame :-), only 5 rows!

In [25]:
# <FILL-IN WITH YOUR CODE>
df.show(5)

+-------------------+----------+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+---------+----+-----+
|               date|Appliances|lights|                T1|              RH_1|                T2|              RH_2|                T3|              RH_3|                T4|              RH_4|                T5|              RH_5|                  T6|               RH_6|                T7|              RH_7|                T8|              RH_8|                T9|              RH_9|               T_out|     

# Data preprocessing [10 marks]

This dataset is nicely prepared for Machine Learning and required very little preprocessing. However, rather than keeping the date as a timestamp, we would like to have some additional columns, including 'day of the year', 'hour', and 'month of the year'. Please use the naming `dayofyear`, `hour` and `month`, respectively.

**Hint**: Of course the SparkSQL library has a function to transform strings with datetime!

Would you like to have any other information from the datetime? Feel free to add other features.

In [27]:
# <FILL-IN WITH YOUR CODE>
from pyspark.sql.functions import col, dayofyear, hour, month

df = df.withColumn("dayofyear", dayofyear(col("date")))
df = df.withColumn("hour", hour(col("date")))
df = df.withColumn("month", month(col("date")))
df.show(5)


+-------------------+----------+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------+----+-----+
|               date|Appliances|lights|                T1|              RH_1|                T2|              RH_2|                T3|              RH_3|                T4|              RH_4|                T5|              RH_5|                  T6|               RH_6|                T7|              RH_7|                T8|              RH_8|                T9|              RH_9|               T_out|       Press_mm_hg|      

When you dataframe `df` has the additional columns, please remove the column `date`:

In [28]:
# <FILL-IN WITH YOUR CODE>
df=df.drop('date')
df.show(5)

+----------+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------+----+-----+
|Appliances|lights|                T1|              RH_1|                T2|              RH_2|                T3|              RH_3|                T4|              RH_4|                T5|              RH_5|                  T6|               RH_6|                T7|              RH_7|                T8|              RH_8|                T9|              RH_9|               T_out|       Press_mm_hg|             RH_out|           Windspeed|     

In [29]:
test("date" in df.columns, False, "Column date has not been remove!")
test("hour" in df.columns, True, "The hour hasn't been added")
test("dayofyear" in df.columns, True, "The dayofyear hasn't been added")
test("month" in df.columns, True, "The month hasn't been added")

1 test passed.
1 test passed.
1 test passed.
1 test passed.


Check the schema of your dataframe:

In [30]:
# <FILL-IN WITH YOUR CODE>
df.printSchema()

root
 |-- Appliances: string (nullable = true)
 |-- lights: string (nullable = true)
 |-- T1: string (nullable = true)
 |-- RH_1: string (nullable = true)
 |-- T2: string (nullable = true)
 |-- RH_2: string (nullable = true)
 |-- T3: string (nullable = true)
 |-- RH_3: string (nullable = true)
 |-- T4: string (nullable = true)
 |-- RH_4: string (nullable = true)
 |-- T5: string (nullable = true)
 |-- RH_5: string (nullable = true)
 |-- T6: string (nullable = true)
 |-- RH_6: string (nullable = true)
 |-- T7: string (nullable = true)
 |-- RH_7: string (nullable = true)
 |-- T8: string (nullable = true)
 |-- RH_8: string (nullable = true)
 |-- T9: string (nullable = true)
 |-- RH_9: string (nullable = true)
 |-- T_out: string (nullable = true)
 |-- Press_mm_hg: string (nullable = true)
 |-- RH_out: string (nullable = true)
 |-- Windspeed: string (nullable = true)
 |-- Visibility: string (nullable = true)
 |-- Tdewpoint: string (nullable = true)
 |-- rv1: string (nullable = true)
 |-- rv2

Dammit, all the input features have been inferred as strings rather than numeric values. This is because we read the data from a CSV file with the data in between quotes.

Your task now is to transform that into numerical values. All of the features are actually numeric, so you could cast all of them. You are recommended to use functions like `cast` and `col` to do this. You could try to leave out the datetime columns we created, but it's fine if you transform them to float.

In [31]:
# <FILL-IN WITH YOUR CODE>
from pyspark.sql.functions import col

datetime_columns = ["timestamp", "dayofyear", "hour", "month"]
all_columns = df.columns

numeric_columns = [col_name for col_name in all_columns if col_name not in datetime_columns]

for col_name in numeric_columns:
    df = df.withColumn(col_name, col(col_name).cast("double"))


In [32]:
#print the schema again.
# <FILL-IN WITH YOUR CODE>
df.printSchema()

root
 |-- Appliances: double (nullable = true)
 |-- lights: double (nullable = true)
 |-- T1: double (nullable = true)
 |-- RH_1: double (nullable = true)
 |-- T2: double (nullable = true)
 |-- RH_2: double (nullable = true)
 |-- T3: double (nullable = true)
 |-- RH_3: double (nullable = true)
 |-- T4: double (nullable = true)
 |-- RH_4: double (nullable = true)
 |-- T5: double (nullable = true)
 |-- RH_5: double (nullable = true)
 |-- T6: double (nullable = true)
 |-- RH_6: double (nullable = true)
 |-- T7: double (nullable = true)
 |-- RH_7: double (nullable = true)
 |-- T8: double (nullable = true)
 |-- RH_8: double (nullable = true)
 |-- T9: double (nullable = true)
 |-- RH_9: double (nullable = true)
 |-- T_out: double (nullable = true)
 |-- Press_mm_hg: double (nullable = true)
 |-- RH_out: double (nullable = true)
 |-- Windspeed: double (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- Tdewpoint: double (nullable = true)
 |-- rv1: double (nullable = true)
 |-- rv2

## Split data into training and test sets [10 marks]

Our final data preparation step is to split our dataset into training and test sets. We will train and tune our model on the training set, and then see how well we do in the test.

Your task is to split the dataframe `df` into 70% for training and 30% for test. Please use the same random seed.

In [34]:
# <FILL-IN WITH YOUR CODE>
# Define the seed
seed = 56

# Split the DataFrame
train_df, test_df = df.randomSplit([0.7, 0.3], seed=seed)

print("Number of rows in training set:", train_df.count())
print("Number of rows in test set:", test_df.count())

Number of rows in training set: 13941
Number of rows in test set: 5794


Even though we have fixed the random seed, you will not get the exact same split. 

Note that this is the simplest way of validating your results. You may want to carry out a [k-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) and split the dataset into *k* folds, and build and test *k* models. We will do later cross validation but for parameter tuning! not to validate our approach!

## Data visualisation [10 marks]

Before applying any machine learning algorithm, it is a good practice to try to visualise your data. For example, we could see how much energy is spent in appliances depending on the month. 

In [ ]:
# create a variable `hist_elect` that contains the histogram of total energy consumed by the Appliances, grouped by month
# <FILL-IN WITH YOUR CODE>

In [ ]:
(x_values, y_values) = zip(*hist_elect)
plt.bar(x_values, y_values)
plt.title('Histogram of repetitions (up to 10)')
plt.xlabel('Month')
plt.ylabel('Sum of Appliance Energy')
plt.show()

As we don't seem to have all the data from the 1st of January, well, we have less consumption in that particular months, so probably the month is not a good feature, don't you think? we are going to remove it from the dataframe and the training and test data frames too!

In [ ]:
# update the variables `df`, `train` and `test`, removing the column 'month'
# <FILL-IN WITH YOUR CODE>

You could do other plots to understand better the data and practice with Spark :-). This is a good opportunity to practice with the DataFrame API, although if you are running this on Databricks, some plots can be generated very easily by yourself.

In [ ]:
# <FILL-IN WITH YOUR CODE>

## Create a Pipeline with Spark ML [20 marks]

As you know, we can't feed the data frame directly to a machine learning algorithm, as we need to put all the input features as an Array, and indicate which one is the output feature (in our case, the 'Appliances' column!).

We will put together a simple Pipeline with the following stages:

- VectorAssembler: To combine all the input columns into a single vector column (i.e. all the columns but the 'Appliances' one.
- Learning algorithm: I feel like using Gradient-Boosted Trees [GBTs](https://en.wikipedia.org/wiki/Gradient_boosting) for this example, but feel free to use anything else.
- CrossValidator: I will use cross validation to tune the parameters of the GBT model. Yes, this can be added as part of a pipeline!  This is going to change the way we access the best model later!  

Step 1: Create the `VectorAssembler`:

In [ ]:
from pyspark.ml.feature import VectorAssembler

# <FILL-IN WITH YOUR CODE>

Step 2: Create an instance of `GBTRegressor` in which you don't indicate any parameters but the class label (i.e. `labelCol`) to 'Appliances'.

In [ ]:
from pyspark.ml.regression import GBTRegressor

#name the variable `gbt` 
# <FILL-IN WITH YOUR CODE>

Step 3: Create a `CrossValidator` for `gbt`.

You can explore the parameters you like for GBT. Full documentation [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.GBTRegressor.html). I would suggest to create a 'grid' for at least the depth of the tree and the number of iterations. Don't investigate more than 4-8 combinations! :-)

Import the right libraries:

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
# define a variable `paramGrid` with some parameters, e.g. for maxDepth, range [5,8], and for maxIter [10,20]

# <FILL-IN WITH YOUR CODE>

Create a RegressionEvaluator that uses the [Root Mean Squared Error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation) as our performance metric. Import the right package first:

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# create a variable `evaluator` with a rmse as metric for the prediction column given by `gbt`.
# <FILL-IN WITH YOUR CODE>

We can now create a CrossValidator `cv` that uses the `gbt` as estimator, as well as the evaluator and grid we defined above.

In [ ]:
# <FILL-IN WITH YOUR CODE>

Step 4. Create a Pipeline (`pipeline`) that contains the two stages: `vectorAssembler` and `cv`

In [ ]:
from pyspark.ml import Pipeline

# <FILL-IN WITH YOUR CODE>

Step 5. Finally, `fit` the model, and store in a `pipelineModel` variable. This might take quite a bit of time!

In [ ]:
# <FILL-IN WITH YOUR CODE>

It could be a good idea to save this to disk, in case it takes too long, so we can read it later.

In [ ]:
pipelineModel.save("Model-lab4")

## Evaluate the results [10 marks]

To obtain the predictions in the test set, apply the method `transform()` of the trained pipeline on the test DataFrame! This will not apply the cross-validation of course!

In [ ]:
# create a variable `predictions`:

# <FILL-IN WITH YOUR CODE>

It is easier to view the results when we limit the columns displayed to:

- `Appliances`: the consumption of the Appliances in Wh
- `prediction`: our predicted prediction

Find a way to show the output only for these two features (only for the first 5 rows):

In [ ]:
# <FILL-IN WITH YOUR CODE>

Are these results any good? Let's compute the RMSE using the evaluator we created before! Store the result in a `rmse` variable.

In [ ]:
# <FILL-IN WITH YOUR CODE>

In [ ]:
print(rmse)

Seems a bit high?  Well, this number is closer to what it is reported in the original paper with RF (RMSE around 69).

But maybe you can investigate a bit more if you can improve that. Can you find out the importance of the features from the GBTs?  

You first need to find out the best model!! In the way we trained the pipeline, you can find the trained cvModel as one of the stages of the `pipelineModel`:

In [ ]:
cvModel = pipelineModel.stages[1]

In [ ]:
cvModel.bestModel

Check the feature importances of that `cvModel`:

In [ ]:
cvModel.bestModel.featureImportances

Uhm, looks like our model gave feature #28 quite a bit of importance, which is the hour of the day if I recall correctly.  Features #25 and #26 were random, and the model noticed that 26 was completely useless, but gave some importance to 'rv1'. GBTs perform somehow an implicit feature selection, so those low importance features won't affect much their performance, but I wonder if we could just remove low importance features?

*Task*: Create a list of those features with less than for example 0.05. 

In [ ]:
pairs = cvModel.bestModel.featureImportances

In [ ]:
# create a list `to_remove` that contains the feature names that must be removed because their confidence is less than 0.05 
# <FILL-IN WITH YOUR CODE>

In [ ]:
print(to_remove)

## Removing low importance features [10 marks]

Check the current schema of the training data:

In [ ]:
# <FILL-IN WITH YOUR CODE>

You don't need to remove the columns from train and test partitions as you won't be able to re-train the pipeline without modifying the VectorAssemble. You simply need to indicate the columns you want to use when creating a new `vectorAssembler2`:

In [ ]:
# <FILL-IN WITH YOUR CODE>

and create a new pipeline, `pipeline2`, with that new `vectorAssemble2`: 

In [ ]:
# <FILL-IN WITH YOUR CODE>

And `fit` the new pipeline:

In [ ]:
# <FILL-IN WITH YOUR CODE>

Finally, make predictions and compute the error:

In [ ]:
# <FILL-IN WITH YOUR CODE>

In [ ]:
print(rmse)

Check the features of the best model and feature importances:

In [ ]:
cvModel2 = pipelineModel2.stages[1]

In [ ]:
cvModel2.bestModel# 

In [ ]:
cvModel2.bestModel.featureImportances

In my case, I got the same performance as before, but this time my model only considered 3 input features! This might not have made our model more precise (Because GBTs already ignored those features), but makes it more interpretable!

# Improving further your model [20 marks]

There might be many ways to improve the results we obtained here. 

A few ideas for you to think about:

- Parameter tuning: I have used a relatively small set of parameters, and I haven't investigated what happened in training and test, is there overfitting of the training? would we be able to use a large number of trees?
- The features of this dataset are numerical, are there other classifiers that may be more appropriate than GBTs?
- We haven't really done any careful pre-processing of the data. Are there outliers or noise that might be having an impact in the results? 
- Do we need any normalisation?

In [ ]:
# <FILL-IN WITH YOUR CODE>